## This notebook is for scraping data from capology

### Copy and paste body element HTML of salary page from Capology in a pdf and upload it here

In [5]:
pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 5.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [231]:
import fitz  # PyMuPDF for PDF extraction
import pandas as pd
from bs4 import BeautifulSoup

# Load PDF and extract text
pdf_path = ""  # Update with the pdf that consists the body element html

html_content = ""
with fitz.open(pdf_path) as doc:
    for page in doc:
        html_content += page.get_text("text")

# Validate extracted content
if "<html" not in html_content.lower():
    print("Error: Extracted text does not seem to contain valid HTML.")
else:
    print("HTML successfully extracted from PDF.")

# Parse extracted HTML using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all tables with id="table"
tables = soup.find_all("table", {"id": "table"})

if not tables:
    print("Error: Could not find any tables with id='table'")
else:
    print(f"Found {len(tables)} tables in the extracted HTML.")

# Initialize a list to store all extracted data
all_players_data = []

# Function to clean salary fields
def clean_salary(value):
    """
    Removes unnecessary spaces and joins multi-line salary values.
    Ensures numbers appear correctly in a single line.
    """
    return value.replace("\n", "").replace("  ", " ").strip()

# Loop through each table and extract data
for idx, salary_table in enumerate(tables):
    print(f"Processing table {idx+1}/{len(tables)}...")

    # Extract rows dynamically
    rows = []
    for row in salary_table.find("tbody").find_all("tr"):
        cols = [col.get_text(strip=True) for col in row.find_all("td")]

        # Apply salary cleaning function to relevant columns
        cols = [clean_salary(col) for col in cols]

        rows.append(cols)

    # Define the standardized column names based on the provided image
    standard_columns = [
        "PLAYER", "GROSS P/W (GBP)", "GROSS P/Y (GBP)", "ADJ. GROSS (GBP)", 
        "POS.", "AGE", "COUNTRY", "CLUB"
    ]

    # Ensure all rows have exactly 8 columns; adjust dynamically
    for row in rows:
        while len(row) < len(standard_columns):
            row.append("")  # Pad missing values

        if len(row) > len(standard_columns):
            row = row[:len(standard_columns)]  # Trim extra columns

    # Create DataFrame for the table and add league/season columns
    df_table = pd.DataFrame(rows, columns=standard_columns)
    df_table["League"] = "Premier League"
    df_table["Season"] = "2017"

    # Append to master list
    all_players_data.append(df_table)

# Combine all tables into one DataFrame
df_final = pd.concat(all_players_data, ignore_index=True)

# Save to CSV
df_final.to_csv("Premier_League_Salaries17.csv", index=False, encoding="utf-8")

print(f"CSV saved successfully! Total rows: {df_final.shape[0]} 🚀")


HTML successfully extracted from PDF.
Found 8 tables in the extracted HTML.
Processing table 1/8...
Processing table 2/8...
Processing table 3/8...
Processing table 4/8...
Processing table 5/8...
Processing table 6/8...
Processing table 7/8...
Processing table 8/8...
CSV saved successfully! Total rows: 735 🚀
